# Basin analysis
Code to calculate autocorrelations and cross-correlations for data within a specific basin or set of basins


In [1]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as plt
import scipy
import statsmodels.api as sm

In [2]:
#define an e-folding time function that takes an autocorrelation or cross-correlation vector as its input

#the function requires the values of autocorrelation or crosscorrelation and the length of the time series used
# to calculate those values

#the function returns the e-folding value, the value that fell below e, and the degrees of freedom

def e_fold(corrTS,lenTS):
    #start from the max value in the first year
    maxVal = np.max(corrTS[:12])
    maxValLoc = np.argmax(corr[:12])
    for i,val in enumerate(corrTS[maxValLoc:]):
        if val < maxVal/np.e: #limit the values of correlation or cross-corr to the first year for max value
            Te = i
            val_Te = val
            break
    dof = (lenTS*1)/(2*Te)
    return Te, val_Te, dof, maxVal, maxValLoc

In [3]:
gapFills = ['clim','glws2'] #gapFill as either 'clim' or 'glws2'

variables = ['e_gleam_mm_mon','glws2_gwater_mm','grace_lwe_cm','grun_ens_runoff_mm_day',
             'prec_fldas_mm_day','smroot_gleam_mm3_mm3','smsurf_gleam_mm3_mm3','lai_gimms4g_m2_m2']
variable2s = ['e_gleam_mm_mon','glws2_gwater_mm','grace_lwe_cm','grun_ens_runoff_mm_day',
              'prec_fldas_mm_day','smroot_gleam_mm3_mm3','smsurf_gleam_mm3_mm3','lai_gimms4g_m2_m2']

basins = ['LIMPOPO','ORANGE','NILE','JUBBA (also GENALE WENZ)']

bName=[];v1=[];v2=[];vcc=[];gFill=[];lens=[];
Te=[];eVal=[];eDoF=[];sig=[];mxV=[];mxVloc=[]
for gapFill in gapFills:
    for basin in basins:
        for variable in variables:
            for variable2 in variable2s:
                #read basin dataframe
                path = './Basin_data/'+variable+'_'+basin+'.csv'
                path2 = './Basin_data/'+variable2+'_'+basin+'.csv'
                glwsPath = './Basin_data/glws2_tws_anom_mm_'+basin+'.csv'

                bDF = pd.read_csv(path)
                bDF2 = pd.read_csv(path2)
                glwsDF = pd.read_csv(glwsPath)

                #plot climatology
                plt.figure()
                bDF.iloc[:,1:].mean(0).plot(title='Climatology of '+variable+'\nin '+basin+' basin')
                plt.savefig('./Figures/Climatologies/'+variable+'_'+basin+'.png')
                plt.close()

                bDFclim = bDF.iloc[:,1:].mean(0)
                bDFclim = bDFclim.reset_index().rename(columns={'index':'Month',0:'value'})
                bDF2clim = bDF2.iloc[:,1:].mean(0)
                bDF2clim = bDF2clim.reset_index().rename(columns={'index':'Month',0:'value'})

                glwsDFclim = glwsDF.iloc[:,1:].mean(0)
                glwsDFclim = glwsDFclim.reset_index().rename(columns={'index':'Month',0:'value'})

                #Calculate anomalies by remove monthly climatology from each month
                bDF.iloc[:,1:] = bDF.iloc[:,1:] - bDF.iloc[:,1:].mean(0)
                bDF2.iloc[:,1:] = bDF2.iloc[:,1:] - bDF2.iloc[:,1:].mean(0)
                glwsDF.iloc[:,1:] = glwsDF.iloc[:,1:] - glwsDF.iloc[:,1:].mean(0)
                #reshape the data
                bDF = bDF.melt(id_vars='Year',var_name='Month')
                bDF2 = bDF2.melt(id_vars='Year',var_name='Month')
                glwsDF = glwsDF.melt(id_vars='Year',var_name='Month')
                #create a datetime index
                dtInd = pd.to_datetime(bDF.Year.astype(int).astype(str) + bDF['Month'], format='%Y%b')
                bDF['date'] = dtInd
                dtInd2 = pd.to_datetime(bDF2.Year.astype(int).astype(str) + bDF2['Month'], format='%Y%b')
                bDF2['date'] = dtInd2
                dtIndglws = pd.to_datetime(glwsDF.Year.astype(int).astype(str) + glwsDF['Month'], format='%Y%b')
                glwsDF['date'] = dtIndglws          
                bDF=bDF.set_index('date')
                bDF2=bDF2.set_index('date')
                glwsDF=glwsDF.set_index('date')

                if (variable=='grun_ens_runoff_mm_day')&(variable2=='grun_ens_runoff_mm_day'):
                    dtInd = dtInd[(dtInd.dt.year>1981)] #limit to satellite era to be more similar to other vars   
                    dtInd2 = dtInd2[(dtInd2.dt.year>1981)]
                if variable=='grace_lwe_cm':
                    if gapFill == 'clim':
                        #replace missing values with climatology (which is = 0 in anomaly space)
                        bDF.loc[np.isnan(bDF['value']),'value'] = 0 
                    elif gapFill == 'glws2':
                        dt_t = np.isin(dtInd,dtIndglws)
                        bDF.loc[(np.isnan(bDF['value']))&(list(dt_t)),'value'] = \
                            glwsDF.loc[(np.isnan(bDF['value']))&(list(dt_t)),'value'].values/10 #convert to cm
                        dtInd = dtInd[(dtInd.dt.year>2002)]
                    else:
                        print('not a valid gap fill option')
                        break
                    dtInd = dtInd[(dtInd.dt.year<=2022)]
                if variable2=='grace_lwe_cm':
                    if gapFill == 'clim':
                        #replace missing values with climatology (which is = 0 in anomaly space)
                        bDF2.loc[np.isnan(bDF2['value']),'value'] = 0
                    elif gapFill == 'glws2':
                        dt_t = np.isin(dtInd2,dtIndglws)
                        bDF2.loc[(np.isnan(bDF2['value']))&(list(dt_t)),'value'] = \
                            glwsDF.loc[(np.isnan(bDF2['value']))&(list(dt_t)),'value'].values/10 #convert to cm
                        dtInd2 = dtInd2[(dtInd2.dt.year>2002)]#glws v2 not available in 2002
                    else:
                        print('not a valid gap fill option')
                        break
                    dtInd = dtInd[(dtInd.dt.year<=2022)]

                #choose only values that exist in both datasets to calculate the cross correlation
                dts = set(dtInd).intersection(dtInd2)
                bDF=bDF.loc[list(dts)]
                bDF2=bDF2.loc[list(dts)]

                #reorder the data chronologically
                bDF = bDF.sort_values(by='date')
                bDF2 = bDF2.sort_values(by='date')

                #Calculate the autocorrelation
                if variable == variable2:
                    anoms = bDF.value.values
                    anoms = anoms-np.nanmean(anoms)
                    std = np.std(anoms)
                    corr = sm.tsa.stattools.ccf(anoms/std,anoms/std,adjusted=True,fft=False)
                    plt.figure()
                    plt.plot(corr[:9])
                    plt.title(variable+' autocorrelation out to 9 months')
                    plt.savefig('./Figures/crosscorrelations_individual_vars/'+variable+'_'+basin+'.png')
                    plt.close()
                else:
                    anoms = bDF.value.values
                    anoms2 = bDF2.value.values
                    anoms = anoms-np.nanmean(anoms)
                    anoms2 = anoms2-np.nanmean(anoms2)
                    std = np.std(anoms)
                    std2 = np.std(anoms2)
                    corr = sm.tsa.stattools.ccf(anoms/std,anoms2/std2,adjusted=True,fft=False)
                    plt.figure()
                    plt.plot(corr[:9])
                    plt.title(variable+' lags \n'+variable2+'\ncross-correlation out to 9 months')
                    plt.savefig('./Figures/crosscorrelations_individual_vars/'+variable+'_'+variable2+'_'+basin+'.png')
                    plt.close()
                
                sig90 = scipy.stats.norm.ppf((1 + 0.90)/2)/np.sqrt(len(anoms))
                #sig90 = scipy.stats.norm.ppf(0.90, loc=np.mean(corr[:48]), scale=np.std(corr[:48]))
                
                Te_t,eVal_t,eDoF_t, mxV_t, mxVloc_t = e_fold(corr,len(anoms))
                
                sig = np.append(sig,1*(sig90<=corr[:9]),0)
                vcc = np.append(vcc,corr[:9],0)
                v1 = np.append(v1,np.repeat(variable,9),0)
                v2 = np.append(v2,np.repeat(variable2,9),0)
                bName = np.append(bName,np.repeat(basin,9),0)
                gFill = np.append(gFill,np.repeat(gapFill,9),0)
                lens = np.append(lens,np.repeat(len(anoms),9),0)
                Te = np.append(Te, np.repeat(Te_t,9),0)
                eVal = np.append(eVal, np.repeat(eVal_t,9),0)
                eDoF = np.append(eDoF, np.repeat(eDoF_t,9),0)
                mxV = np.append(mxV, np.repeat(mxV_t,9),0)
                mxVloc = np.append(mxVloc, np.repeat(mxVloc_t,9),0)
                
df =pd.DataFrame(data={'basin':bName,'var1':v1,'var2':v2,'cross_corr':vcc,'gapFill':gFill,
                       'n':lens,'Te':Te,'eDoF':eDoF,'90pctSig':sig,'maxCorr':mxV,'maxCorrLag',mxVloc})
path = './crosscorr_dfs/basin_cc.pkl'
df.to_pickle(path)

In [4]:
#First calculate the e-folding time of the cross-correlations 
#then write it to a new array and plot the cross-correlations

plt.ioff()

colors = {
'e_gleam_mm_mon':'limegreen',
'glws2_gwater_mm':'navy',
'grace_lwe_cm':'red',
'grun_ens_runoff_mm_day':'blue',
'prec_fldas_mm_day':'cornflowerblue',
'smroot_gleam_mm3_mm3':'saddlebrown',
'smsurf_gleam_mm3_mm3':'peru',
'lai_gimms4g_m2_m2':'green'
}

Te=[],
for igapFill in gapFills:
    for basin in basins:
        for variable in variables:
            plt.figure()
            for variable2 in variable2s:
                if variable==variable2:continue
                corrTS = df[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable2)].cross_corr.values
                sigCorr = df.loc[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable2),'90pctSig'].values
                plt.plot(corrTS,label=str(variable2),color=colors[variable2])
                if np.sum(sigCorr>0):
                    plt.scatter(np.where(sigCorr==1)[0],corrTS[sigCorr==1],color=colors[variable2])
            plt.legend()
            plt.title(basin+'\n'+variable+' lags [X]')
            plt.savefig('./Figures/crosscorrelations_all_vars/'+variable+'_allVars_'+basin+'_'+igapFill+'.png')
            plt.close()

/var/folders/pk/_ym78shs7tx06w00zbcwxwgc0000gp/T/ipykernel_62390/997312057.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


In [5]:
basins = ['LIMPOPO','ORANGE','NILE','JUBBA (also GENALE WENZ)']

colors = {
'JUBBA (also GENALE WENZ)':'red',
'NILE':'cornflowerblue',
'ORANGE':'peru',
'LIMPOPO':'green'
}

for igapFill in gapFills:
    for variable in variables:
        plt.figure()
        for basin in basins:
            corrTS = df[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable)].cross_corr.values
            sigCorr = df.loc[(df.gapFill==igapFill)&(df.basin==basin)&(df.var1==variable)&(df.var2==variable),'90pctSig'].values
            plt.plot(corrTS,label=str(basin),color=colors[basin])
            if np.sum(sigCorr>0):
                    plt.scatter(np.where(sigCorr==1)[0],corrTS[sigCorr==1],color=colors[basin])
            plt.legend()
            plt.title(variable+' autocorrelation')
            plt.savefig('./Figures/autocorrelations_all_basins/'+variable+'_'+igapFill+'_allbasins.png')
            plt.close()

In [24]:
for igapFill in gapFills:
    for variable in variables:
        plt.figure()
        eFold = df.loc[(df.gapFill==igapFill)&(df.var1==variable)&(df.var2==variable),['basin','Te']]
        eFold.drop_duplicates().plot.bar(x='basin',y='Te')
        plt.title(variable+' autocorrelation e-folding time (months)')
        plt.tight_layout()
        plt.savefig('./Figures/e_Folding/eFolding_autocorrelation_'+variable+'_'+igapFill+'_allbasins.png')      
        plt.close()